In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
import face_recognition
import numpy as np
import os

  
known_face_encodings = []
known_face_names = []
video_capture = None


def load_known_faces():
    folder_path = "C:/Users/Enter Computer/known_image"
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)
            if encoding:
                known_face_encodings.append(encoding[0])
                known_face_names.append(os.path.splitext(filename)[0])

def release_camera():
    global video_capture
    if video_capture:
        video_capture.release()
        cv2.destroyAllWindows()
        video_capture = None

def show_custom_message(message, bg_color, fg_color):
    popup = tk.Toplevel()
    popup.title("Message")
    popup.configure(bg=bg_color)
    popup.geometry("400x120+900+500")  # Positioned lower-right, slightly up and left
    label = tk.Label(popup, text=message, bg=bg_color, fg=fg_color,
                     font=("Arial", 14, "bold"), wraplength=380, justify="center")
    label.pack(expand=True, fill='both', padx=10, pady=10)
    popup.after(3000, popup.destroy)

def show_custom_error(message):
    show_custom_message(message, "#2c2f33", "#ff4c4c")  # dark bg, red text

def show_custom_success(message):
    show_custom_message(message, "#2c2f33", "#6cc644")  # dark bg, green success

# ============ Register Function ============

def capture_face_for_registration(username):
    global video_capture
    video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    attempts = 3  # Reduced to 3 for faster check
    success = False

    for _ in range(attempts):
        ret, frame = video_capture.read()
        if not ret:
            continue

        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(rgb_small)

        if face_encodings:
            encoding = face_encodings[0]

            # Check if face already registered
            if known_face_encodings:
                distances = face_recognition.face_distance(known_face_encodings, encoding)
                if min(distances) < 0.5:
                    show_custom_error("❌ This face is already registered. Registration failed.")
                    release_camera()
                    return

            known_face_encodings.append(encoding)
            known_face_names.append(username)
            image_path = f"C:/Users/Enter Computer/known_image/{username}.jpg"
            cv2.imwrite(image_path, frame)
            show_custom_success("✅ Registered successfully.")
            success = True
            break

    release_camera()

    if not success:
        show_custom_error("❌ No face detected. Try again.")

def register():
    username = username_entry.get().strip()
    if not username:
        show_custom_error("Please enter a username.")
        return
    if username in known_face_names:
        show_custom_error("Username already exists!")
        return
    capture_face_for_registration(username)


def login():
    username = username_entry.get().strip()
    if not username:
        show_custom_error("Please enter a username.")
        return

    if username not in known_face_names:
        show_custom_error("Username not found!")
        return

    global video_capture
    video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    index = known_face_names.index(username)
    target_encoding = known_face_encodings[index]

    matched = False
    attempts = 3  # Reduced to 3 for faster check

    for _ in range(attempts):
        ret, frame = video_capture.read()
        if not ret:
            continue

        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_encodings = face_recognition.face_encodings(rgb_small)

        for face_encoding in face_encodings:
            distance = face_recognition.face_distance([target_encoding], face_encoding)[0]
            if distance < 0.45:
                matched = True
                break

        if matched:
            break

    release_camera()

    if matched:
        show_custom_success(f"✅ Welcome, {username}!")
    else:
        show_custom_error("❌ Face does not match the username.")


app = tk.Tk()
app.title("Face Recognition Login/Register")
app.geometry("400x200+600+300")  # Smaller screen and more centered
app.configure(bg="#1e1e1e")

tk.Label(app, text="Username:", font=("Arial", 14), bg="#1e1e1e", fg="white").pack(pady=20)
username_entry = tk.Entry(app, font=("Arial", 14), bg="#2c2f33", fg="white", insertbackground="white")
username_entry.pack(ipadx=10, ipady=5)

tk.Button(app, text="Register", font=("Arial", 13), command=register,
          bg="#7289da", fg="white", activebackground="#5b6eae").pack(pady=15)

tk.Button(app, text="Login", font=("Arial", 13), command=login,
          bg="#99aab5", fg="black", activebackground="#7a8a94").pack()

load_known_faces()
app.mainloop()
